<h1>Welcome to my TxMM project</h1> This project is about the missing of Hebe and Sanne in October 2022 in the Netherlands.

By: Roel Duijsings

Install needed packages:

In [9]:
#! pip install datasets transformers

<h2>Load the training data</h2>

<h3>Rewrite training data: replace tweet_id by tweet_text</h3>
Create a training_data_{limit}.csv file

In [ ]:
from snscrape.modules.twitter import TwitterTweetScraper
import pandas as pd
from preprocess import replacePlaceholders

limit = 10000
tweets = []
dict_emotions = {'anger': 0,'fear':1,'joy':2,'love':3,'neutral':4,'sadness':5}
with open("EmotioNL_tweets.txt") as f:
    next(f)  # skip header
    i = 0
    # For every tweet in the file...
    for line in f:
        if i >= limit:
            break

        line_splitted = line.split()
        tweet_id = line_splitted[1][2:-2]
        category = line_splitted[-1]
        label = dict_emotions[category]
        try:
            # ...scrape that tweet from Twitter using tweet_id
            for tweet in TwitterTweetScraper(tweet_id).get_items():
                text = replacePlaceholders(tweet.content)
                # TODO: how to correctly save the data?
                data_point = {"label": label, "text": text}
                tweets.append(data_point)
                # print(tweet_id, data_point)
        except:
            print("Something went wrong when scraping the tweets.")
            pass
        i += 1

df = pd.DataFrame(tweets)
df.index.name = "idx"
df.to_csv(f"training_data_{len(df.index)}.csv")  # save as .csv file
print("- - - Finished loading training data - - -")


<h3> Train the model on the training data</h3>

Initialize tokenizer and model

In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "pdelobelle/robbert-v2-dutch-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast = True)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)


loading configuration file config.json from cache at C:\Users\roell/.cache\huggingface\hub\models--pdelobelle--robbert-v2-dutch-base\snapshots\94c743437a36425637fcf29d3c939c813da0398c\config.json
Model config RobertaConfig {
  "_name_or_path": "pdelobelle/robbert-v2-dutch-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 40000
}

loading file vocab.json from cache at C:\Users\roell/.

In [3]:
tokenizer("Hallo allemaal")

{'input_ids': [0, 7751, 422, 2], 'attention_mask': [1, 1, 1, 1]}

Load training data

In [18]:
from datasets.load import load_dataset


def tokenize_function(tweet):
    return tokenizer(tweet["text"], padding="max_length", truncation=True)


data_file = "training_data_749.csv"
train_dataset = load_dataset("csv", data_files=data_file)
train_dataset_tokenized = train_dataset.map(tokenize_function, batched=True)

small_train_dataset = train_dataset_tokenized["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = train_dataset_tokenized["test"].shuffle(seed=42).select(range(1000))

Using custom data configuration default-151a34b549011ef0


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 298.70it/s]
Generating train split: 0 examples [00:00, ? examples/s]c:\Users\roell\Documents\Master AI\Text and Multimedia Mining v2\Research paper\TxMM-research-project\.venv\lib\site-packages\datasets\download\streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to C:/Users/roell/.cache/huggingface/datasets/csv/default-151a34b549011ef0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00,  4.51ba/s]


IndexError: Index 999 out of range for dataset of size 749.

In [5]:
train_dataset["train"][0:3]


{'idx': [0, 1, 2],
 'label': [0, 0, 0],
 'text': ['@USER Maandagmorgen en extra druk😣Dat is nu voordeel van niet meer werken op maandag al dit nieuws lezen!',
  '@USER Echt...zweet en shag..en dat een hele dag😡Je zal ermee op kantoor zitten😝',
  '@USER @USER @USER D66Pechtold zei 1e BELANGRIJKSTE!poging😂dream on Pechtold gelul id ruimte! PVV id 2e grtste!wat e kleuterklasje.pesten uitsluiten z nie 2017']}

Train the model on the training data

In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized["train"],  # TODO: deze dataset komt niet goed door! check format!
    tokenizer=tokenizer,
)

trainer.train()


100%|██████████| 18/18 [09:33<00:00, 29.59s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 18/18 [09:33<00:00, 31.87s/it]

{'train_runtime': 573.5755, 'train_samples_per_second': 0.235, 'train_steps_per_second': 0.031, 'train_loss': 0.31736066606309676, 'epoch': 3.0}


TrainOutput(global_step=18, training_loss=0.31736066606309676, metrics={'train_runtime': 573.5755, 'train_samples_per_second': 0.235, 'train_steps_per_second': 0.031, 'train_loss': 0.31736066606309676, 'epoch': 3.0})

In [15]:
evaluation_dataset = load_dataset("csv", data_files="training_data_10.csv")
trainer.evaluate(evaluation_dataset["train"]["text"])

Using custom data configuration default-212d9a7667c1efdf
Found cached dataset csv (C:/Users/roell/.cache/huggingface/datasets/csv/default-212d9a7667c1efdf/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 333.28it/s]
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8


AttributeError: 'list' object has no attribute 'keys'

<h2>Load the testdata</h2>

In [7]:
from preprocess import preprocess
query = "(hebe) lang:nl until:2022-10-21 since:2022-10-17"
maxTweets = 100000
data = preprocess(query, maxTweets)

In [8]:
data.to_csv(f"tweets_{len(data.index)}.csv")